# Welcome to Miniverse

**Build emergent agent-based simulations powered by LLMs.**

This notebook shows you what Miniverse does through a simple Mars habitat simulation. You'll see:

- **3 agents** (Commander, Engineer, Scientist) coordinate on Mars base operations
- **LLM-driven decisions** - agents plan, communicate, and adapt
- **Emergent behavior** - watch coordination emerge from individual goals
- **Full observability** - inspect plans, memories, and reasoning

## Prerequisites

Before running this notebook, configure your LLM provider:

```bash
export LLM_PROVIDER=openai
export LLM_MODEL=gpt-5-nano
export OPENAI_API_KEY=your_key
```

Let's get started!

## Step 1: Import and Check Setup

First, let's verify your LLM configuration is working.

In [36]:
import os
import asyncio
from datetime import datetime, timezone

# Check LLM configuration
provider = os.getenv('LLM_PROVIDER')
model = os.getenv('LLM_MODEL')

print('=== LLM Configuration ===')
if not provider or not model:
    print('❌ LLM not configured')
    print('   Set LLM_PROVIDER, LLM_MODEL, and API key environment variables')
else:
    print(f'✅ Provider: {provider}')
    print(f'✅ Model: {model}')
    print(f'✅ API Key: {"Set" if os.getenv(f"{provider.upper()}_API_KEY") else "Missing"}')

=== LLM Configuration ===
✅ Provider: openai
✅ Model: gpt-5-nano
✅ API Key: Set


## Step 2: Import Miniverse Components

Miniverse has a modular architecture:

- **WorldState** - Tracks resources, agents, environment
- **AgentProfile** - Defines agent identity, goals, relationships
- **SimulationRules** - Your custom physics (deterministic updates)
- **AgentCognition** - How agents think (executor, planner, memory)
- **Orchestrator** - Coordinates everything

In [37]:
from miniverse import (
    Orchestrator, AgentProfile, AgentStatus, WorldState,
    ResourceState, EnvironmentState, SimulationRules,
    Stat, AgentAction
)
from miniverse.cognition import AgentCognition, LLMExecutor, LLMPlanner, Scratchpad

print('✅ Miniverse components imported')
print('\nKey classes available:')
print('   • WorldState - simulation state container')
print('   • AgentProfile - agent identity and goals')
print('   • SimulationRules - custom physics interface')
print('   • AgentCognition - LLM-powered decision making')
print('   • Orchestrator - main simulation coordinator')

✅ Miniverse components imported

Key classes available:
   • WorldState - simulation state container
   • AgentProfile - agent identity and goals
   • SimulationRules - custom physics interface
   • AgentCognition - LLM-powered decision making
   • Orchestrator - main simulation coordinator


## Step 3: Define the Physics

Miniverse separates **deterministic physics** from **emergent cognition**.

Your physics rules define:
- How resources change each tick
- How agent actions affect the world
- Natural degradation and constraints

Agents then make decisions using LLMs within this physics framework.

In [38]:
class MarsHabitatRules(SimulationRules):
    """Simple Mars base: manage power, conduct research, handle maintenance."""
    
    def apply_tick(self, state, tick):
        updated = state.model_copy(deep=True)
        
        # Get current resources
        power = updated.resources.get_metric('power', default=100, unit='kWh')
        research = updated.resources.get_metric('research_progress', default=0, unit='%')
        maintenance = updated.resources.get_metric('system_health', default=100, unit='%')
        
        # Agent actions affect resources
        for agent in updated.agents:
            if agent.activity == 'generate_power':
                power.value = min(100, float(power.value) + 15)
            elif agent.activity == 'research':
                research.value = min(100, float(research.value) + 10)
                power.value = max(0, float(power.value) - 8)  # Research uses power
            elif agent.activity == 'maintenance':
                maintenance.value = min(100, float(maintenance.value) + 12)
        
        # Natural degradation each tick
        power.value = max(0, float(power.value) - 5)  # Base consumption
        maintenance.value = max(0, float(maintenance.value) - 3)  # Systems degrade
        
        updated.tick = tick
        return updated
    
    def validate_action(self, action, state):
        return True  # Accept all actions for this simple example

print('✅ Mars habitat physics defined')
print('\nPhysics rules:')
print('   Action effects:')
print('   • generate_power: +15 kWh')
print('   • research: +10% progress, -8 kWh')
print('   • maintenance: +12% system health')
print('\n   Natural degradation:')
print('   • Power: -5 kWh per tick (base consumption)')
print('   • System health: -3% per tick (wear and tear)')

✅ Mars habitat physics defined

Physics rules:
   Action effects:
   • generate_power: +15 kWh
   • research: +10% progress, -8 kWh
   • maintenance: +12% system health

   Natural degradation:
   • Power: -5 kWh per tick (base consumption)
   • System health: -3% per tick (wear and tear)


## Step 4: Create the World State

The world state defines the starting conditions:
- Initial resource levels (power at 60%, research at 25%, systems at 70%)
- Which agents exist in the simulation
- Environment configuration (we're using abstract tier for this simple example)

We're starting with a challenging situation - power and systems both below optimal levels!

In [39]:
world_state = WorldState(
    tick=0,
    timestamp=datetime.now(timezone.utc),
    environment=EnvironmentState(metrics={}),
    resources=ResourceState(metrics={
        'power': Stat(value=60, unit='kWh', label='Power Reserve'),
        'research_progress': Stat(value=25, unit='%', label='Research Progress'),
        'system_health': Stat(value=70, unit='%', label='System Health')
    }),
    agents=[
        AgentStatus(agent_id='commander', display_name='Commander Liu'),
        AgentStatus(agent_id='engineer', display_name='Engineer Kim'),
        AgentStatus(agent_id='scientist', display_name='Dr. Patel')
    ]
)

print('✅ World state initialized')
print('\nInitial conditions:')
print(f'   Tick: {world_state.tick}')
print('\n   Resources:')
for key, stat in world_state.resources.metrics.items():
    status = '⚠️' if (key == 'power' and stat.value < 80) or (key == 'system_health' and stat.value < 80) else '✅'
    print(f'   {status} {stat.label}: {stat.value}{stat.unit}')
print('\n   Agents:')
for agent in world_state.agents:
    print(f'   • {agent.display_name} (id: {agent.agent_id})')
print('\n   Challenge: Power and system health need attention!')

✅ World state initialized

Initial conditions:
   Tick: 0

   Resources:
   ⚠️ Power Reserve: 60kWh
   ✅ Research Progress: 25%
   ⚠️ System Health: 70%

   Agents:
   • Commander Liu (id: commander)
   • Engineer Kim (id: engineer)
   • Dr. Patel (id: scientist)

   Challenge: Power and system health need attention!


## Step 5: Define Agent Profiles

Each agent has:
- **Identity** - name, age, background
- **Role** - their job on the Mars base
- **Personality** - how they approach decisions
- **Skills** - what they're good at
- **Goals** - what they're trying to achieve
- **Relationships** - how they relate to other agents

These profiles inform the LLM's decision-making - agents act according to their character!

In [40]:
agents = {
    'commander': AgentProfile(
        agent_id='commander',
        name='Commander Liu',
        age=42,
        background='15 years military + 5 years NASA mission experience',
        role='mission_commander',
        personality='strategic, calm under pressure, prioritizes crew safety',
        skills={'leadership': 'expert', 'decision_making': 'expert'},
        goals=['Ensure crew safety', 'Complete mission objectives', 'Maintain team morale'],
        relationships={'engineer': 'trusts technical judgment', 'scientist': 'values research input'}
    ),
    'engineer': AgentProfile(
        agent_id='engineer',
        name='Engineer Kim',
        age=35,
        background='Aerospace engineer, 8 years ISS systems experience',
        role='systems_engineer',
        personality='detail-oriented, proactive, safety-focused',
        skills={'systems': 'expert', 'power_management': 'expert', 'maintenance': 'expert'},
        goals=['Keep systems operational', 'Optimize power usage', 'Prevent failures'],
        relationships={'commander': 'reports to', 'scientist': 'coordinates with'}
    ),
    'scientist': AgentProfile(
        agent_id='scientist',
        name='Dr. Patel',
        age=38,
        background='Astrobiologist, 10 years Mars research',
        role='lead_scientist',
        personality='curious, methodical, mission-driven',
        skills={'research': 'expert', 'analysis': 'expert'},
        goals=['Advance Mars research', 'Make discoveries', 'Publish findings'],
        relationships={'commander': 'collaborates with', 'engineer': 'relies on for equipment'}
    )
}

print('✅ Agent profiles created')
print('\n=== Meet the Crew ===')
for agent_id, profile in agents.items():
    print(f'\n👤 {profile.name}')
    print(f'   Role: {profile.role}')
    print(f'   Age: {profile.age}')
    print(f'   Personality: {profile.personality}')
    print(f'   Skills: {", ".join(f"{k}" for k in profile.skills.keys())}')
    print(f'   Goals:')
    for goal in profile.goals:
        print(f'     • {goal}')
    print(f'   Relationships:')
    for other, rel in profile.relationships.items():
        other_name = agents[other].name if other in agents else other
        print(f'     • {other_name}: {rel}')

✅ Agent profiles created

=== Meet the Crew ===

👤 Commander Liu
   Role: mission_commander
   Age: 42
   Personality: strategic, calm under pressure, prioritizes crew safety
   Skills: leadership, decision_making
   Goals:
     • Ensure crew safety
     • Complete mission objectives
     • Maintain team morale
   Relationships:
     • Engineer Kim: trusts technical judgment
     • Dr. Patel: values research input

👤 Engineer Kim
   Role: systems_engineer
   Age: 35
   Personality: detail-oriented, proactive, safety-focused
   Skills: systems, power_management, maintenance
   Goals:
     • Keep systems operational
     • Optimize power usage
     • Prevent failures
   Relationships:
     • Commander Liu: reports to
     • Dr. Patel: coordinates with

👤 Dr. Patel
   Role: lead_scientist
   Age: 38
   Personality: curious, methodical, mission-driven
   Skills: research, analysis
   Goals:
     • Advance Mars research
     • Make discoveries
     • Publish findings
   Relationships:
     

## Step 6: Configure Agent Cognition

This is where the LLM magic happens! Each agent gets a **cognition bundle**:

- **Executor** - Makes immediate decisions ("What action should I take right now?")
- **Planner** - Creates multi-step plans ("What's my agenda for the day?")
- **Scratchpad** - Working memory for tracking commitments and state

We're using `LLMExecutor` and `LLMPlanner`, which call your configured LLM to make intelligent, context-aware decisions.

We also give each agent **role instructions** - these are injected into the LLM prompts to guide their behavior.

In [41]:
# Create cognition modules for each agent
cognition_map = {
    'commander': AgentCognition(
        executor=LLMExecutor(),
        planner=LLMPlanner(),
        scratchpad=Scratchpad()
    ),
    'engineer': AgentCognition(
        executor=LLMExecutor(),
        planner=LLMPlanner(),
        scratchpad=Scratchpad()
    ),
    'scientist': AgentCognition(
        executor=LLMExecutor(),
        planner=LLMPlanner(),
        scratchpad=Scratchpad()
    )
}

# Role-specific instructions (injected into LLM prompts)
agent_prompts = {
    'commander': '''You are Commander Liu on a Mars habitat.

Your responsibilities:
- Oversee base operations and crew coordination
- Make strategic decisions about priorities
- Communicate with team via brief updates

Available actions: generate_power, research, maintenance, coordinate, rest

Current priorities: Safety > Systems > Research''',
    
    'engineer': '''You are Engineer Kim maintaining Mars habitat systems.

Your responsibilities:
- Monitor power levels and system health
- Perform maintenance when needed
- Generate power when reserves are low
- Advise commander on technical issues

Available actions: generate_power, maintenance, rest

Keep power above 40 kWh and systems above 60%.''',
    
    'scientist': '''You are Dr. Patel, lead scientist on Mars.

Your responsibilities:
- Conduct research experiments
- Balance research with base needs
- Communicate findings and coordinate with crew

Available actions: research, rest

Research is important but requires power - coordinate with engineer.'''
}

print('✅ Agent cognition configured')
print('\n=== Cognition Architecture ===')
print('\nEach agent has:')
print('   🧠 LLMExecutor - Makes immediate action decisions via LLM')
print('   📋 LLMPlanner - Generates multi-step plans via LLM')
print('   📝 Scratchpad - Working memory for state tracking')
print('\nAgent instructions configured:')
for agent_id in agent_prompts:
    agent_name = agents[agent_id].name
    lines = agent_prompts[agent_id].strip().split('\n')
    print(f'   • {agent_name}: {lines[0]}')

✅ Agent cognition configured

=== Cognition Architecture ===

Each agent has:
   🧠 LLMExecutor - Makes immediate action decisions via LLM
   📋 LLMPlanner - Generates multi-step plans via LLM
   📝 Scratchpad - Working memory for state tracking

Agent instructions configured:
   • Commander Liu: You are Commander Liu on a Mars habitat.
   • Engineer Kim: You are Engineer Kim maintaining Mars habitat systems.
   • Dr. Patel: You are Dr. Patel, lead scientist on Mars.


## Step 7: Run the Simulation

Now we bring it all together with the **Orchestrator**.

Each tick, the orchestrator:
1. Applies physics rules (resources change)
2. Agents observe the world
3. Agents plan/decide using LLMs
4. Agents take actions
5. Actions are processed
6. Memories are stored

Let's run 5 ticks and watch the agents coordinate!

In [42]:
# Create and configure orchestrator
orchestrator = Orchestrator(
    world_state=world_state,
    agents=agents,
    world_prompt='',
    agent_prompts=agent_prompts,
    simulation_rules=MarsHabitatRules(),
    agent_cognition=cognition_map,
    llm_provider=provider,
    llm_model=model
)

print('🚀 Starting Mars habitat simulation...')
print(f'   Provider: {provider}')
print(f'   Model: {model}')
print(f'   Duration: 5 ticks')
print('\nWatch agents coordinate to manage power, research, and maintenance!')
print('=' * 60)

result = await orchestrator.run(num_ticks=5)

print('=' * 60)
print('\n✅ Simulation complete!')
print(f'   Run ID: {result["run_id"]}')
print(f'   Final tick: {result["final_state"].tick}')

🚀 Starting Mars habitat simulation...
   Provider: openai
   Model: gpt-5-nano
   Duration: 5 ticks

Watch agents coordinate to manage power, research, and maintenance!
Starting simulation run b3b5ab65-e4eb-4833-a3b6-64667770c39b
Agents: 3, Ticks: 5

=== Tick 1/5 ===
  [Physics] Applying deterministic rules for tick 1...
  [Physics] ✓ Physics applied
  [Commander Liu] Building perception...
  [Engineer Kim] Building perception...
  [Dr. Patel] Building perception...
  [Engineer Kim] Choosing action via executor...
  [Commander Liu] Choosing action via executor...
  [Dr. Patel] Choosing action via executor...
  [Engineer Kim] ✓ Got action: work
LLM schema validation failed for AgentAction (attempt 1/3).
    - communication: Input should be a valid dictionary [type=dict_type] | received="Hi Kim, coordinating power optimization to ensure sufficient power for a 60-...
LLM retry 2/3 for AgentAction; attempting schema correction.
  [Commander Liu] ✓ Got action: communicate
  [Dr. Patel] ✓ Go

ValidationError: 1 validation error for AgentMemory
importance
  Input should be a valid integer [type=int_type, input_value=None, input_type=NoneType]
    For further information visit https://errors.pydantic.dev/2.12/v/int_type

## Step 8: Inspect the Results

Let's see how the world changed during the simulation.

We'll compare initial vs final resource levels and see what each agent ended up doing.

In [ ]:
final = result['final_state']

print('=== SIMULATION RESULTS ===')
print(f'\nTicks elapsed: {final.tick}')

print('\n--- Resource Changes ---')
for key in ['power', 'research_progress', 'system_health']:
    initial = world_state.resources.metrics[key]
    final_stat = final.resources.metrics[key]
    change = float(final_stat.value) - float(initial.value)
    arrow = '↑' if change > 0 else '↓' if change < 0 else '→'
    print(f'\n{final_stat.label}:')
    print(f'   Initial: {initial.value}{initial.unit}')
    print(f'   Final:   {final_stat.value}{final_stat.unit}')
    print(f'   Change:  {arrow} {abs(change):.1f}{final_stat.unit}')

print('\n--- Final Agent Activities ---')
for agent in final.agents:
    agent_profile = agents[agent.agent_id]
    print(f'   • {agent.display_name}: {agent.activity or "idle"}')

=== SIMULATION RESULTS ===

Ticks elapsed: 5

--- Resource Changes ---

Power Reserve:
   Initial: 60kWh
   Final:   34.0kWh
   Change:  ↓ 26.0kWh

Research Progress:
   Initial: 25%
   Final:   36%
   Change:  ↑ 11.0%

System Health:
   Initial: 70%
   Final:   65.0%
   Change:  ↓ 5.0%

--- Final Agent Activities ---
   • Commander Liu: finalizing crew briefing and assigning tasks; roles confirmed within 25-minute window
   • Engineer Kim: performing proactive maintenance on power_subsystems; stabilizing power reserve; load management readiness
   • Dr. Patel: synthesizing Mars soil data findings; research progress updated to 36%


## Step 9: Peek Under the Hood - Agent Reasoning

The magic of Miniverse is **full observability**. Let's inspect what agents were thinking!

We can retrieve actions from any tick and see:
- What action the agent chose
- Why they chose it (LLM reasoning)
- What they communicated to others

In [ ]:
run_id = result['run_id']

# Get actions from final tick
final_tick = final.tick
tick_actions = await orchestrator.persistence.get_actions(run_id, tick=final_tick)

print(f'=== AGENT REASONING (Tick {final_tick}) ===')
print('\nWhat were agents thinking in the final tick?\n')

if not tick_actions:
    print('⚠️  No actions found for this tick.')
    print('   (This might happen if actions weren\'t saved during the simulation)')
else:
    for action in tick_actions:
        agent_name = agents[action.agent_id].name
        print(f'🤖 {agent_name}')
        print(f'   Action: {action.action_type}')
        print(f'   Reasoning: {action.reasoning}')
        if action.communication:
            # communication is a Dict with message content
            if isinstance(action.communication, dict):
                msg = action.communication.get('message', str(action.communication))
                print(f'   💬 Communication: "{msg}"')
            else:
                print(f'   💬 Communication: "{action.communication}"')
        print()

=== AGENT REASONING (Tick 5) ===

What were agents thinking in the final tick?



## Step 10: Inspect Agent Memories

Agents remember what they observe each tick. These memories inform future decisions.

Let's look at what the Engineer remembers - did they notice the power crisis?

In [ ]:
# Get engineer's memories (last 10)
engineer_memories = await orchestrator.persistence.get_recent_memories(
    run_id, 
    agent_id='engineer',
    limit=10
)

print(f'=== ENGINEER\'S MEMORY LOG ===')
print(f'\nTotal memories retrieved: {len(engineer_memories)}')

if not engineer_memories:
    print('\n⚠️  No memories found for this agent.')
    print('   (Memories are created as agents observe the world each tick)')
else:
    print(f'\nRecent observations (last {min(5, len(engineer_memories))}):\n')
    
    # Show most recent 5
    for i, mem in enumerate(list(reversed(engineer_memories))[:5], 1):
        print(f'{i}. [Tick {mem.tick}]')
        # Truncate long memories for readability
        content = mem.content if len(mem.content) <= 120 else mem.content[:120] + '...'
        print(f'   {content}')
        if mem.tags:
            print(f'   Tags: {", ".join(mem.tags)}')
        print()

## Step 11: Action Statistics

Let's analyze what actions agents took across all ticks.

This shows us the emergent behavior - how did agents coordinate without explicit teamwork code?

In [ ]:
from collections import defaultdict, Counter

# Get all actions across all ticks
all_actions = []
for tick in range(1, final.tick + 1):
    tick_actions = await orchestrator.persistence.get_actions(run_id, tick=tick)
    all_actions.extend(tick_actions)

print('=== ACTION STATISTICS ===')
print(f'\nTotal actions: {len(all_actions)}')

# Actions by agent
agent_actions = defaultdict(list)
for action in all_actions:
    agent_actions[action.agent_id].append(action.action_type)

print('\n--- Actions by Agent ---')
for agent_id, action_list in agent_actions.items():
    agent_name = agents[agent_id].name
    action_counts = Counter(action_list)
    print(f'\n{agent_name}:')
    for action_type, count in action_counts.most_common():
        print(f'   • {action_type}: {count}x')

# Communication stats
communications = [a for a in all_actions if a.communication]
print(f'\n--- Communication ---')
print(f'Messages sent: {len(communications)}')
if communications:
    print('\nSample messages:')
    for i, action in enumerate(communications[:3], 1):
        agent_name = agents[action.agent_id].name
        msg = action.communication[:80] + '...' if len(action.communication) > 80 else action.communication
        print(f'   {i}. {agent_name}: "{msg}"')

## 💡 What You Just Saw

**Miniverse simulated 5 hours on Mars with 3 LLM-powered agents**

### Each Tick:
1. ⚙️ **Physics applied** - power drained, systems degraded, actions took effect
2. 👁️ **Agents observed** - saw resource levels, other agents' activities
3. 🧠 **Agents planned** - LLM generated multi-step plans based on role and situation
4. ✅ **Agents acted** - chose actions aligned with plans and goals
5. 💾 **Memories stored** - observations saved for future reasoning

### Key Features:
- **Emergent coordination** - no hardcoded teamwork, agents adapted based on situation
- **LLM reasoning** - decisions explained in natural language
- **Memory persistence** - agents remember past events
- **Full observability** - inspect plans, memories, reasoning at any point

### The Setup (Core Pattern):
```python
# 1. Define physics (SimulationRules subclass)
class MarsHabitatRules(SimulationRules):
    def apply_tick(self, state, tick):
        # Your domain logic here
        return updated_state

# 2. Create agents with profiles
agents = {"commander": AgentProfile(...), ...}

# 3. Configure cognition (LLM-powered)
cognition = {"commander": AgentCognition(
    executor=LLMExecutor(),
    planner=LLMPlanner()
)}

# 4. Run simulation
orchestrator = Orchestrator(...)
result = await orchestrator.run(num_ticks=5)
```

**That's it!** Physics + profiles + cognition = emergent behavior.

## 🚀 Next Steps

### Explore More Examples
- **`tutorial.ipynb`** - Reference guide for all primitives (Stat, Plan, Memory, etc.)
- **`examples/workshop/`** - Progressive examples:
  - `01_hello_world` - Simplest possible simulation
  - `02_deterministic` - Threshold-based logic (no LLM)
  - `03_llm_single` - Single LLM agent
  - `04_team_chat` - Multi-agent communication
  - `05_stochastic` - Random events + LLM adaptation

### Build Your Own Simulation
1. **Define physics** - subclass `SimulationRules`
2. **Create agents** - profiles with goals and relationships
3. **Choose cognition** - deterministic, LLM, or hybrid
4. **Run and observe** - inspect emergent behavior

### Common Use Cases
- Social simulations (Stanford Generative Agents style)
- Multi-agent systems research
- Game AI and NPC behavior
- Organizational dynamics
- Economic modeling

### What Makes Miniverse Special
- ✅ **Deterministic physics** - controllable, predictable rules
- ✅ **Emergent cognition** - LLM-driven intelligence
- ✅ **Full observability** - inspect everything
- ✅ **Modular design** - swap strategies easily
- ✅ **Production-ready** - persistence, memory, planning all built-in

---

**Ready to build something?** Check out `docs/USAGE.md` for detailed guides!